In [ ]:
# 품번을 폴더명으로 하고 해당 url에서 사진을 224*224로 가져오는 코드
# 품번이 nan값: 폴더명 nan으로 저장 
# 품번에/가 포함되어있는 경우: /를 _로 대체

import os
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import re

# CSV 파일 불러오기
csv_file_path = r'C:\Users\leehy\Desktop\240827_toyproject_khuda\musinsa_image_crawling 2.csv'
df = pd.read_csv(csv_file_path)

# 이미지가 저장될 기본 경로 설정
base_save_path = r'C:\Users\leehy\Desktop\240827_toyproject_khuda\스냅사진'

# 이미지 크기 설정
target_size = (224, 224)

# 허용되지 않는 문자 패턴 정의 (Windows에서 허용되지 않는 문자들)
invalid_char_pattern = r'[\/:*?"<>|]'

# 이미지 전처리 및 저장 함수
def preprocess_and_save_image(url, save_dir, index):
    try:
        # 이미지 다운로드
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        
        # 이미지 크기 조정
        img = img.resize(target_size)
        
        # 이미지 저장 경로 설정
        save_path = os.path.join(save_dir, f'{index}.jpg')
        
        # 이미지 저장
        img.save(save_path)
        
        print(f"Saved image {index} from {url} to {save_path}")
    except Exception as e:
        print(f"Failed to process image from {url}: {e}")

# 스냅사진 열에서 URL과 품번 가져오기
for index, row in df.iterrows():
    품번 = row['품번']  # 각 상품의 품번을 폴더 이름으로 사용
    
    # 품번에서 허용되지 않는 문자 제거
    품번 = re.sub(invalid_char_pattern, '_', str(품번))
    
    image_urls = str(row['스냅사진']).split(',')  # 스냅사진 열의 URL들 (콤마로 분리되어 있다고 가정)
    
    if image_urls == ['nan']:  # 이미지 URL이 없는 경우 건너뜀
        print(f"No image URLs found for 품번: {품번}")
        continue
    
    # 품번 폴더 경로 설정
    save_dir = os.path.join(base_save_path, str(품번))
    os.makedirs(save_dir, exist_ok=True)
    
    # 각 URL에 대해 이미지 다운로드 및 저장
    for i, url in enumerate(image_urls):
        preprocess_and_save_image(url.strip(), save_dir, i+1)

print("All images have been processed and saved.")
